In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Installing libraries

In [ ]:
!pip install textattack
!pip install -U gensim==4.0.0
!pip install python-Levenshtein

     |████████████████████████████████| 358kB 6.6MB/s 
     |████████████████████████████████| 102kB 5.6MB/s 
     |████████████████████████████████| 20.1MB 51.5MB/s 
     |████████████████████████████████| 2.1MB 36.8MB/s 
     |████████████████████████████████| 225kB 38.1MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 337kB 32.4MB/s 
     |████████████████████████████████| 778kB 33.3MB/s 
     |████████████████████████████████| 3.3MB 35.9MB/s 
     |████████████████████████████████| 901kB 33.8MB/s 
     |████████████████████████████████| 112kB 42.2MB/s 
     |████████████████████████████████| 245kB 41.7MB/s 
     |████████████████████████████████| 798kB 41.0MB/s 
     |████████████████████████████████| 983kB 41.1MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 1.2MB 33.3MB/s 
  Created wheel for lru-dict: filename=lru_dict-1.1.7

     |████████████████████████████████| 23.9MB 130kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 51kB 3.0MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149803 sha256=d5593d7a21a4d498f31099f1aa1e8e70212acee7eee2d6c6e07702c142f8138e
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models import Phrases
from gensim.test.utils import get_tmpfile
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.transformations import WordSwap
from textattack.search_methods import GreedySearch, GreedyWordSwapWIR
from textattack.constraints.overlap import LevenshteinEditDistance
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:11<00:00, 40.6MB/s]
textattack: Unzipping file /root/.cache/textattack/tmphblp0k_i.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [ ]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.fasttext import FastText
# model1 = FastText(corpus, sample = 0, sg = 1)

In [ ]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.word2vec import Word2Vec
# model2 = Word2Vec(corpus, sample = 0, sg = 1)

# Loading FastText, Word2Vec and Glove

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
#fname = get_tmpfile("/models/fasttext.model") 
model1 = FastText.load("/content/drive/MyDrive/models/fasttext.model") # Change this location

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.word2vec import Word2Vec
#fname1 = get_tmpfile("/models/word2vec.model") 
model2 = Word2Vec.load("/content/drive/MyDrive/models/word2vec.model") # Change this location

In [ ]:
from gensim.models import KeyedVectors
model3=KeyedVectors.load('/content/drive/MyDrive/models/gloveKeyVectors.kv')

Example demonstration

In [ ]:
print(model1.wv.most_similar('cat'))
print(model2.wv.most_similar('cat'))
print(model3.most_similar('cat'))

[('bobcat', 0.759185791015625), ('dog', 0.7208123803138733), ('foxglove', 0.7110776901245117), ('pig', 0.7088143229484558), ('goose', 0.7069101333618164), ('cats', 0.702387273311615), ('beak', 0.7008379697799683), ('catfish', 0.6987316608428955), ('foxhound', 0.696578860282898), ('catnip', 0.6960986852645874)]
[('dog', 0.7193913459777832), ('squirrel', 0.696162760257721), ('leopardus', 0.6945451498031616), ('prionailurus', 0.688106894493103), ('albino', 0.6880608201026917), ('bird', 0.6836060285568237), ('eared', 0.6803243160247803), ('rhinoceros', 0.6769237518310547), ('hyena', 0.6756084561347961), ('badger', 0.6752869486808777)]
[('cats', 0.7006820440292358), ('dog', 0.698471188545227), ('dogs', 0.6442825198173523), ('tervueren', 0.619421660900116), ('laekenois', 0.5972026586532593), ('lica', 0.5956287980079651), ('feral', 0.5899650454521179), ('goat', 0.5861616134643555), ('fish', 0.5746133327484131), ('malinois', 0.5737614631652832)]


In [ ]:
if 'cat' in model3.key_to_index:
  print('present')

present


# Distilbert Base Cased-QQP

In [ ]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-QQP")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-QQP")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Framing fasttext attack

In [ ]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

In [ ]:
def attack_stats(attack, dataset, num_of_examples=25):
  print(attack)
  num_examples = num_of_examples
  num_remaining_attacks = num_examples
  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

  worklist = deque(range(0, num_examples))
  worklist_tail = worklist[-1]

  attack_log_manager = textattack.loggers.AttackLogManager()

  load_time = time.time()
        
  num_results = 0
  num_failures = 0
  num_successes = 0
  for result in attack.attack_dataset(dataset, indices=worklist):
      result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
      display(HTML(result_html_str))
      attack_log_manager.log_result(result)
            
      if not isinstance(result, textattack.attack_results.SkippedAttackResult):
          pbar.update(1)
      else:
          worklist_tail += 1
          pbar.update(1)
          worklist.append(worklist_tail)

      num_results += 1

      if (type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult):
          num_successes += 1
                
      if type(result) == textattack.attack_results.FailedAttackResult:
          num_failures += 1
      pbar.set_description(
          "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
      )

  pbar.close()

  attack_log_manager.enable_stdout()
  attack_log_manager.log_summary()
  attack_log_manager.flush()
        
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

### FastText, GreedySearch, RepeatModification, StopwordModification

In [ ]:

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("glue", "qqp", "validation") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


textattack: Loading datasets dataset glue, subset qqp, split validation.


Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:01<00:33,  1.41s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:25<04:51, 12.66s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:25<03:09,  8.60s/it]

...

[Succeeded / Failed / Total] 12 / 11 / 25: 100%|██████████| 25/25 [01:22<00:00,  3.29s/it]

[Succeeded / Failed / Total] 12 / 12 / 26: : 26it [01:24,  3.24s/it]

[Succeeded / Failed / Total] 13 / 12 / 27: : 27it [01:27,  3.22s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 12     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 44.44% |
| Attack success rate:          | 52.0%  |
| Average perturbed word %:     | 10.17% |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 69.8   |
+-------------------------------+--------+


textattack: Attack time: 87.03159046173096s


# Framing word2vec attack

In [ ]:
class Swapper_(WordSwap): # For word2vec
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model2.wv.key_to_index:
            for i in range(10):
                if word.lower() in model2.wv.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model2.wv.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model2.wv.most_similar(word)[i][0].capitalize()]
                else:
                    return [model2.wv.most_similar(word)[i][0]]
        else:
            return [word]

### Word2vec, GreedySearch, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:08,  2.91it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:10<01:59,  5.19s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:10<01:18,  3.58s/it]

...

[Succeeded / Failed / Total] 7 / 16 / 25: 100%|██████████| 25/25 [00:41<00:00,  1.68s/it]

[Succeeded / Failed / Total] 7 / 17 / 26: : 26it [00:43,  1.65s/it]

[Succeeded / Failed / Total] 8 / 17 / 27: : 27it [00:43,  1.62s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 62.96% |
| Attack success rate:          | 32.0%  |
| Average perturbed word %:     | 6.44%  |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 38.44  |
+-------------------------------+--------+


textattack: Attack time: 43.721490144729614s


### Framing Glove attack

In [ ]:
from textattack.transformations import WordSwap

class Swapper__(WordSwap): # For glove
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model3.key_to_index:
            for i in range(10):
                if word.lower() in model3.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model3.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model3.most_similar(word)[i][0].capitalize()]
                else:
                    return [model3.most_similar(word)[i][0]]
        else:
            return [word]

### Glove, GreedySearch, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:12,  1.97it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:17<03:16,  8.55s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:17<02:08,  5.86s/it]

...

[Succeeded / Failed / Total] 9 / 14 / 25: 100%|██████████| 25/25 [01:02<00:00,  2.51s/it]

[Succeeded / Failed / Total] 9 / 15 / 26: : 26it [01:04,  2.47s/it]

[Succeeded / Failed / Total] 10 / 15 / 27: : 27it [01:04,  2.40s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 15     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 55.56% |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 7.51%  |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 36.24  |
+-------------------------------+--------+


textattack: Attack time: 64.85688376426697s


### FastText, GreedyWordSwapWIR, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:17,  1.34it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:04<00:48,  2.09s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:04<00:35,  1.63s/it]

...

[Succeeded / Failed / Total] 10 / 13 / 25: 100%|██████████| 25/25 [00:31<00:00,  1.27s/it]

[Succeeded / Failed / Total] 10 / 14 / 26: : 26it [00:33,  1.27s/it]

[Succeeded / Failed / Total] 10 / 15 / 27: : 27it [00:34,  1.28s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 15     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 55.56% |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 11.06% |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 33.28  |
+-------------------------------+--------+


textattack: Attack time: 34.449021100997925s


### Word2Vec, GreedyWordSwapWIR, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:12,  1.89it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:03<00:35,  1.56s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:03<00:27,  1.26s/it]

...

[Succeeded / Failed / Total] 7 / 16 / 25: 100%|██████████| 25/25 [00:26<00:00,  1.05s/it]

[Succeeded / Failed / Total] 7 / 17 / 26: : 26it [00:27,  1.04s/it]

[Succeeded / Failed / Total] 8 / 17 / 27: : 27it [00:27,  1.03s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 62.96% |
| Attack success rate:          | 32.0%  |
| Average perturbed word %:     | 7.13%  |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 29.92  |
+-------------------------------+--------+


textattack: Attack time: 27.909448385238647s


### Glove, GreedyWordSwapWIR RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:14,  1.65it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:03<00:44,  1.92s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:04<00:33,  1.51s/it]

...

[Succeeded / Failed / Total] 9 / 14 / 25: 100%|██████████| 25/25 [00:29<00:00,  1.17s/it]

[Succeeded / Failed / Total] 9 / 15 / 26: : 26it [00:30,  1.17s/it]

[Succeeded / Failed / Total] 10 / 15 / 27: : 27it [00:31,  1.16s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 15     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 55.56% |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 9.11%  |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 29.52  |
+-------------------------------+--------+


textattack: Attack time: 31.40264391899109s


### FastText, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [ ]:
from textattack.search_methods import ParticleSwarmOptimization


# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:03<01:14,  3.11s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [01:08<13:06, 34.21s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [01:09<08:26, 23.03s/it]

...

[Succeeded / Failed / Total] 12 / 11 / 25: 100%|██████████| 25/25 [03:55<00:00,  9.42s/it]

[Succeeded / Failed / Total] 12 / 12 / 26: : 26it [04:02,  9.34s/it]

[Succeeded / Failed / Total] 13 / 12 / 27: : 27it [04:09,  9.24s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 12     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 44.44% |
| Attack success rate:          | 52.0%  |
| Average perturbed word %:     | 11.16% |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 298.68 |
+-------------------------------+--------+


textattack: Attack time: 249.42525839805603s


### Word2Vec, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:20,  1.18it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:31<06:00, 15.69s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:31<03:53, 10.60s/it]

...

[Succeeded / Failed / Total] 7 / 16 / 25: 100%|██████████| 25/25 [01:54<00:00,  4.57s/it]

[Succeeded / Failed / Total] 7 / 17 / 26: : 26it [01:56,  4.47s/it]

[Succeeded / Failed / Total] 8 / 17 / 27: : 27it [01:57,  4.36s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 62.96% |
| Attack success rate:          | 32.0%  |
| Average perturbed word %:     | 6.98%  |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 223.56 |
+-------------------------------+--------+


textattack: Attack time: 117.86090326309204s


### Glove, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:16,  1.48it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:53<10:15, 26.77s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:54<06:36, 18.01s/it]

...

[Succeeded / Failed / Total] 9 / 14 / 25: 100%|██████████| 25/25 [03:11<00:00,  7.65s/it]

[Succeeded / Failed / Total] 9 / 15 / 26: : 26it [03:13,  7.45s/it]

[Succeeded / Failed / Total] 10 / 15 / 27: : 27it [03:14,  7.19s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 15     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 55.56% |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 10.09% |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 220.48 |
+-------------------------------+--------+


textattack: Attack time: 194.25151252746582s


### FastText, GreedyWordSwapWIR, PartsofSpeech

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:21,  1.13it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:03<00:45,  2.00s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:05<00:37,  1.72s/it]

...

[Succeeded / Failed / Total] 9 / 14 / 25: 100%|██████████| 25/25 [00:32<00:00,  1.29s/it]

[Succeeded / Failed / Total] 9 / 15 / 26: : 26it [00:33,  1.29s/it]

[Succeeded / Failed / Total] 10 / 15 / 27: : 27it [00:34,  1.27s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 15     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 55.56% |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 16.49% |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 34.72  |
+-------------------------------+--------+


textattack: Attack time: 34.37549948692322s


### Word2Vec, GreedyWordSwapWIR, PartsofSpeech

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:12,  1.89it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:03<00:35,  1.55s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:03<00:27,  1.24s/it]

...

[Succeeded / Failed / Total] 8 / 15 / 25: 100%|██████████| 25/25 [00:28<00:00,  1.12s/it]

[Succeeded / Failed / Total] 8 / 16 / 26: : 26it [00:29,  1.12s/it]

[Succeeded / Failed / Total] 9 / 16 / 27: : 27it [00:30,  1.12s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 59.26% |
| Attack success rate:          | 36.0%  |
| Average perturbed word %:     | 11.47% |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 31.96  |
+-------------------------------+--------+


textattack: Attack time: 30.134086847305298s


### Glove, GreedyWordSwapWIR, PartsofSpeech

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:20,  1.18it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:04<00:50,  2.18s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:05<00:37,  1.72s/it]

...

[Succeeded / Failed / Total] 7 / 16 / 25: 100%|██████████| 25/25 [00:36<00:00,  1.47s/it]

[Succeeded / Failed / Total] 7 / 17 / 26: : 26it [00:38,  1.48s/it]

[Succeeded / Failed / Total] 8 / 17 / 27: : 27it [00:39,  1.47s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 62.96% |
| Attack success rate:          | 32.0%  |
| Average perturbed word %:     | 8.52%  |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 30.2   |
+-------------------------------+--------+


textattack: Attack time: 39.65823698043823s


### FastText, ParticleSwarmOptimization, PartsofSpeech

In [ ]:
from textattack.constraints.grammaticality import PartOfSpeech
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:08<03:35,  8.97s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [01:20<15:22, 40.12s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [01:39<12:06, 33.02s/it]

...

[Succeeded / Failed / Total] 10 / 13 / 25: 100%|██████████| 25/25 [07:33<00:00, 18.13s/it]

[Succeeded / Failed / Total] 10 / 14 / 26: : 26it [07:54, 18.23s/it]

[Succeeded / Failed / Total] 11 / 14 / 27: : 27it [07:57, 17.69s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 14     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 51.85% |
| Attack success rate:          | 44.0%  |
| Average perturbed word %:     | 12.41% |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 464.88 |
+-------------------------------+--------+


textattack: Attack time: 477.7385129928589s


### Word2Vec, ParticleSwarmOptimization, PartsofSpeech

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:02<00:59,  2.50s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:55<10:36, 27.69s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:56<06:51, 18.70s/it]

...

[Succeeded / Failed / Total] 9 / 14 / 25: 100%|██████████| 25/25 [05:12<00:00, 12.49s/it]

[Succeeded / Failed / Total] 9 / 15 / 26: : 26it [05:27, 12.59s/it]

[Succeeded / Failed / Total] 10 / 15 / 27: : 27it [05:29, 12.22s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 15     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 55.56% |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 9.71%  |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 362.32 |
+-------------------------------+--------+


textattack: Attack time: 329.87921953201294s


### Glove, ParticleSwarmOptimization, PartsofSpeech

In [ ]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [ ]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:07<02:58,  7.44s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [01:26<16:29, 43.02s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [01:27<10:38, 29.02s/it]

...

[Succeeded / Failed / Total] 7 / 16 / 25: 100%|██████████| 25/25 [08:24<00:00, 20.19s/it]

[Succeeded / Failed / Total] 7 / 17 / 26: : 26it [08:46, 20.25s/it]

[Succeeded / Failed / Total] 8 / 17 / 27: : 27it [08:47, 19.54s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 62.96% |
| Attack success rate:          | 32.0%  |
| Average perturbed word %:     | 6.97%  |
| Average num. words per input: | 22.3   |
| Avg num queries:              | 309.88 |
+-------------------------------+--------+


textattack: Attack time: 527.585437297821s
